# Coursework - 9

## Problem 9.1

### R-5.1
Execute the experiment from Code Fragment 5.1 and compare the results
on your system to those we report in Code Fragment 5.2.

In [1]:
import sys

data = []
n = 26  # Example value for n, you can set it as needed

for k in range(n):
    a = len(data)  # Number of elements in the list
    b = sys.getsizeof(data)  # Actual size in bytes
    print(f"Length: {a:03d}; Size in bytes: {b:1.4f}")
    data.append(None)  # Increase length by one


Length: 000; Size in bytes: 56.0000
Length: 001; Size in bytes: 88.0000
Length: 002; Size in bytes: 88.0000
Length: 003; Size in bytes: 88.0000
Length: 004; Size in bytes: 88.0000
Length: 005; Size in bytes: 120.0000
Length: 006; Size in bytes: 120.0000
Length: 007; Size in bytes: 120.0000
Length: 008; Size in bytes: 120.0000
Length: 009; Size in bytes: 184.0000
Length: 010; Size in bytes: 184.0000
Length: 011; Size in bytes: 184.0000
Length: 012; Size in bytes: 184.0000
Length: 013; Size in bytes: 184.0000
Length: 014; Size in bytes: 184.0000
Length: 015; Size in bytes: 184.0000
Length: 016; Size in bytes: 184.0000
Length: 017; Size in bytes: 248.0000
Length: 018; Size in bytes: 248.0000
Length: 019; Size in bytes: 248.0000
Length: 020; Size in bytes: 248.0000
Length: 021; Size in bytes: 248.0000
Length: 022; Size in bytes: 248.0000
Length: 023; Size in bytes: 248.0000
Length: 024; Size in bytes: 248.0000
Length: 025; Size in bytes: 312.0000


### R-5.2
In Code Fragment 5.1, we perform an experiment to compare the length of
a Python list to its underlying memory usage. Determining the sequence
of array sizes requires a manual inspection of the output of that program.
Redesign the experiment so that the program outputs only those values of
k at which the existing capacity is exhausted. For example, on a system
consistent with the results of Code Fragment 5.2, your program should
output that the sequence of array capacities are 0, 4, 8, 16, 25, ....


In [2]:
import sys

data = []
n = 78  # Example value for n, you can set it as needed

previous_size = 0  # Initialize previous size
for k in range(n):
    a = len(data)  # Number of elements in the list
    b = sys.getsizeof(data)  # Actual size in bytes
    if b != previous_size:
        print(f"Length: {a:03d}; Size in bytes: {b:1.4f}")
        previous_size = b  # Update previous size only when it changes
    data.append(None)  # Increase length by one

Length: 000; Size in bytes: 56.0000
Length: 001; Size in bytes: 88.0000
Length: 005; Size in bytes: 120.0000
Length: 009; Size in bytes: 184.0000
Length: 017; Size in bytes: 248.0000
Length: 025; Size in bytes: 312.0000
Length: 033; Size in bytes: 376.0000
Length: 041; Size in bytes: 472.0000
Length: 053; Size in bytes: 568.0000
Length: 065; Size in bytes: 664.0000
Length: 077; Size in bytes: 792.0000


*The results are different on my machine, here python seems to be using different sizes for the list, it starts already with smaller size for length of 0 and size is always smaller than the values in the example.*

### R-5.3
Modify the experiment from Code Fragment 5.1 in order to demonstrate
that Python’s list class occasionally shrinks the size of its underlying array
when elements are popped from a list.


In [3]:
import sys

n = 78
data = [None for i in range(n)]  # Initialize with 1000 elements

previous_size = 0  # Initialize previous size
for k in range(n):
    a = len(data)  # Number of elements in the list
    b = sys.getsizeof(data)  # Actual size in bytes
    if b != previous_size:
        print(f"Length: {a:03d}; Size in bytes: {b:1.4f}")
        previous_size = b  # Update previous size only when it changes
    data.pop()  # Decrease length by one

Length: 078; Size in bytes: 792.0000
Length: 045; Size in bytes: 504.0000
Length: 027; Size in bytes: 344.0000
Length: 017; Size in bytes: 248.0000
Length: 011; Size in bytes: 184.0000
Length: 007; Size in bytes: 152.0000
Length: 005; Size in bytes: 120.0000
Length: 001; Size in bytes: 88.0000


## Problem 9.2
Review the list implementation in Algorithm 2. Extend it by implementing setitem (self, k, value), insert(self, k, value) and remove(self, k) methods.

In [4]:
import ctypes


class MyArray2:
    def __init__(self):
        self._n = 0
        self._capacity = 1
        self._A = self._make_array(self._capacity)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        if not 0 <= k < self._n:
            raise IndexError("invalid index")
        return self._A[k]

    def __setitem__(self, k, obj):
        if not 0 <= k < self._n:
            raise IndexError("invalid index")
        self._A[k] = obj

    def insert(self, k, obj):
        if not 0 <= k <= self._n:
            raise IndexError("invalid index")

        if self._n == self._capacity:
            self._resize(2 * self._capacity)

        for j in range(self._n, k, -1):
            self._A[j] = self._A[j - 1]

        self._A[k] = obj
        self._n += 1

    def remove(self, k):
        if not 0 <= k < self._n:
            raise IndexError("invalid index")

        for j in range(k, self._n - 1):
            self._A[j] = self._A[j + 1]

        self._n -= 1

        # if self._n == self._capacity // 2:   -- bad because of trashing so resizing every time on border
        if self._capacity > 1 and self._n <= self._capacity // 4:
            self._resize(self._capacity // 2)

    def append(self, obj):
        if self._n == self._capacity:
            self._resize(2 * self._capacity)
        self._A[self._n] = obj
        self._n += 1

    def _resize(self, c):
        B = self._make_array(c)
        for k in range(self._n):
            B[k] = self._A[k]
        self._A = B
        self._capacity = c

    def _make_array(self, c):
        return (c * ctypes.py_object)()

## Problem 9.3
Extend the `getitem(self, k)` method such that it can handle negative indices.

In [5]:
class MyArray3:
    def __init__(self):
        self._n = 0
        self._capacity = 1
        self._A = self._make_array(self._capacity)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        if self._n == 0:
            raise IndexError("invalid index, array is empty")
        effective_k = k

        if k < 0:
            effective_k = self._n + k

        if not 0 <= effective_k < self._n:
            raise IndexError("invalid index")

        return self._A[effective_k]

    def __setitem__(self, k, obj):
        if not 0 <= k < self._n:
            raise IndexError("invalid index")
        self._A[k] = obj

    def insert(self, k, obj):
        if not 0 <= k <= self._n:
            raise IndexError("invalid index")

        if self._n == self._capacity:
            self._resize(2 * self._capacity)

        for j in range(self._n, k, -1):
            self._A[j] = self._A[j - 1]

        self._A[k] = obj
        self._n += 1

    def remove(self, k):
        if not 0 <= k < self._n:
            raise IndexError("invalid index")

        for j in range(k, self._n - 1):
            self._A[j] = self._A[j + 1]

        self._n -= 1

        # if self._n == self._capacity // 2:   -- bad because of trashing so resizing every time on border
        if self._capacity > 1 and self._n <= self._capacity // 4:
            self._resize(self._capacity // 2)

    def append(self, obj):
        if self._n == self._capacity:
            self._resize(2 * self._capacity)
        self._A[self._n] = obj
        self._n += 1

    def _resize(self, c):
        B = self._make_array(c)
        for k in range(self._n):
            B[k] = self._A[k]
        self._A = B
        self._capacity = c

    def _make_array(self, c):
        return (c * ctypes.py_object)()

## Problem 9.4
### C-5.15
Implement a pop method for the DynamicArray class, given in Code Fragment 5.3, that removes the last element of the array, and that shrinks the capacity, N, of the array by half any time the number of elements in the array goes below N/4.

In [6]:
class DynamicArray:
    """A dynamic array class akin to a simplified Python list."""

    def __init__(self):
        """Create an empty array."""
        self._n = 0  # count actual elements
        self._capacity = 1  # default array capacity
        self._A = self._make_array(self._capacity)  # low-level array

    def __len__(self):
        """Return number of elements stored in the array."""
        return self._n

    def __getitem__(self, k):
        """Return element at index k."""
        if not 0 <= k < self._n:
            raise IndexError("invalid index")
        return self._A[k]  # retrieve from array

    def append(self, obj):
        """Add object to end of the array."""
        if self._n == self._capacity:  # not enough room
            self._resize(2 * self._capacity)  # so double capacity
        self._A[self._n] = obj
        self._n += 1

    def _resize(self, c):  # nonpublic utitity
        """Resize internal array to capacity c."""
        B = self._make_array(c)  # new (bigger) array
        for k in range(self._n):  # for each existing value
            B[k] = self._A[k]
        self._A = B  # use the bigger array
        self._capacity = c

    def _make_array(self, c):  # nonpublic utitity
        """Return new array with capacity c."""
        return (c * ctypes.py_object)()  # see ctypes documentation

    def pop(self):
        if self._n == 0:
            raise IndexError("pop from empty array")
        item = self._A[self._n - 1]
        self._n -= 1
        if self._n <= self._capacity // 4:
            self._resize(self._capacity // 2)
        return item

### C-5.25
The syntax data.remove(value) for Python list data removes only the first occurrence of element value from the list. Give an implementation of a function, with signature remove all(data, value), that removes all occur- rences of value from the given list, such that the worst-case running time of the function is O(n) on a list with n elements. Not that it is not efficient enough in general to rely on repeated calls to remove.


In [7]:
def remove_all(data, value):
    new_data = []
    for i in range(len(data)):
        if not data[i] == value:
            new_data.append(data[i])
    return new_data

In [8]:
import time

# Test cases
test_data_1 = [1, 2, 3, 4, 2, 5, 2]
test_data_2 = [2, 2, 2, 2, 2]
test_data_3 = [1, 3, 5, 7, 9]
test_data_4 = [1] * 1000000 + [2] * 1000000  # Large input for performance check


# Function to test remove_all
def test_remove_all():
    print("Original:", test_data_1)
    print("After removing 2:", remove_all(test_data_1, 2))

    print("Original:", test_data_2)
    print("After removing 2:", remove_all(test_data_2, 2))

    print("Original:", test_data_3)
    print("After removing 2:", remove_all(test_data_3, 2))

    # Performance check
    start_time = time.time()
    remove_all(test_data_4, 1)
    end_time = time.time()
    print(f"Performance check (O(n)): {end_time - start_time:.4f} seconds")


test_remove_all()

Original: [1, 2, 3, 4, 2, 5, 2]
After removing 2: [1, 3, 4, 5]
Original: [2, 2, 2, 2, 2]
After removing 2: []
Original: [1, 3, 5, 7, 9]
After removing 2: [1, 3, 5, 7, 9]
Performance check (O(n)): 0.1001 seconds


## Problem 9.5
### R-6.1
What values are returned during the following series of stack operations, if executed upon an initially empty stack? \
`push(5)`, `push(3)`, `pop()`, `push(2)`, `push(8)`, `pop()`, `pop()`, `push(9)`, `push(1)`, `pop()`, `push(7)`, `push(6)`, `pop()`, `pop()`, `push(4)`, `pop()`, `pop()`.

5\
53\
5\
52\
528\
52\
5\
59\
591\
59\
597\
5976\
597\
59\
594\
59\
5

### R-6.10
Consider what happens if the loop in the `ArrayQueue._resize` at lines 53–55 of Code Fragment 6.7 had been implemented as:
```python
for k in range(self._size):
    self._data[k] == old[k]       # rather than old[walk]
```
Give a clear explanation of what could go wrong.

In [9]:
# old code
def _resize(self, cap):
    """Resize to a new list of capacity >= len(self)."""
    old = self._data  # Keep track of the existing list
    self._data = [None] * cap  # Allocate list with new capacity
    walk = self._front

    for k in range(self._size):
        self._data[k] = old[walk]  # Intentionally shift indices
        walk = (1 + walk) % len(old)  # Use old size as modulus

    self._front = 0  # Front has been realigned


*If resizing does not start a new list, the queue has empty entries between the previous end and the new resized end because there are many cases where the start is in the middle of the list and after reaching the end, the next entry would be at index 0.*

### R-6.12
What values are returned during the following sequence of deque ADT operations, on initially empty deque?\
`add_first(4)`, `add_last(8)`, `add_last(9)`, `add_first(5)`, `back()`, `delete_first()`, `delete_last()`, `add_last(7)`, `first()`, `last()`, `add_last(6)`, `delete_first()`, `delete_first()`

[4]\
[4,8]\
[4,8,9]\
[5,4,8,9]\
[5,4,8,9]  -> 9\
[4,8,9] -> 5\
[4,8] -> 9\
[4,8,7]
[4,8,7] -> 4\
[4,8,7] -> 7\
[4,8,7,6]
[8,7,6] -> 4\
[7,6] -> 8

### C-6.17
In the previous exercise, we assume that the underlying list is initially empty. Redo that exercise, this time preallocating an underlying list with length equal to the stack’s maximum capacity.

In [10]:
class ArrayStack:
    """LIFO Stack implementation using a Python list as underlying storage."""

    def __init__(self):
        """Create an empty stack."""
        self._data = []  # Nonpublic list instance

    def __len__(self):
        """Return the number of elements in the stack."""
        return len(self._data)

    def is_empty(self):
        """Return True if the stack is empty."""
        return len(self._data) == 0

    def push(self, e):
        """Add element e to the top of the stack."""
        self._data.append(e)  # New item stored at end of list

    def top(self):
        """Return (but do not remove) the element at the top of the stack.

        Raise Empty exception if the stack is empty.
        """
        if self.is_empty():
            raise Exception("Stack is empty")
        return self._data[-1]  # The last item in the list

    def pop(self):
        """Remove and return the element from the top of the stack (i.e., LIFO).

        Raise Empty exception if the stack is empty.
        """
        if self.is_empty():
            raise Exception("Stack is empty")
        return self._data.pop()  # Remove last item from list


In [11]:
# new code implementation
class ArrayStackPrefilled:
    def __init__(self, maxlen=None):
        self._data = [] if maxlen is None else [None] * maxlen
        self._capacity = maxlen
        self._n = 0

    def __len__(self):
        return self._n

    def is_empty(self):
        return self._n == 0

    def push(self, e):
        if self._capacity is not None and self._n == self._capactiy:
            raise Exception("Stack is full!")
        if self._capacity is None and self._n == len(self._data):
            self._resize(2 * len(self._data) if len(self._data) > 0 else 1)
        self._data[self._n] = e
        self._n += 1

    def top(self):
        if self.is_empty():
            raise Exception("Stack is empty")
        return self._data[self._n - 1]  # The last item in the list

    def pop(self):
        if self.is_empty():
            raise Exception("Stack is empty")
        item = self._data[self._n - 1]
        self._data[self._n - 1] = None
        self._n -= 1
        return item

    def _resize(self, new_capacity):
        old_data = self._data
        self._data = [None] * new_capacity
        for k in range(self._n):
            self._data[k] = old_data[k]

## Problem 9.6

Do exercise P-6.34 (evaluation of postfix notation) in Goodrich et al. Hint: use a stack.

### P-6.34 

Implement a program that can input an expression in postfix notation (see Exercise C-6.22) and output its value.

*Postfix notation is an unambiguous way of writing an arithmetic expression without parentheses. It is defined so that if “(exp1) op (exp2)” is a normal, fully parenthesized expression whose operation is op, the postfix version of this is “pexp1 pexp2 op”, where pexp1 is the postfix version of exp1 and pexp2 is the postfix version of exp2 . The postfix version of a single number or variable is just that number or variable. For example, the postfix version of “$((5 + 2) ∗ (8 − 3))/4$” is “$5\, 2 + 8 \,3 − \,∗\, 4 \,/$”. Describe a nonrecursive way of evaluating an expression in postfix notation.*

In [12]:
math_operators = ["+", "-", "*", "/", "^"]


def to_postfix(equation):
    # ( ( 5 + 2 ) * ( 8 - 3 ) ) / 4
    # pexp1 4 /
    # pexp2 pexp3 * 4 /
    # 5 2 + 8 3 - * 4 /
    stack = []
    result = ""
    for char in equation:
        if char == " ":  # Skip spaces in input
            continue
        if char == "(":
            stack.append(char)
        elif char in math_operators:
            stack.append(char)
        elif str.isalnum(char):
            if (
                result and result[-1].isalnum()
            ):  # Add space between consecutive operands
                result += " "
            result += char
        elif char == ")":
            op = next_math_op(stack)
            if op:  # Only add a space if an operator was actually popped
                if (
                    result and result[-1] != " "
                ):  # Ensure space before operator if result isn't empty/ends in space
                    result += " "
                result += (
                    op.strip()
                )  # next_math_op adds a space, strip it here and add space correctly
            # Standard Shunting-Yard pops until the matching '('
            if stack and stack[-1] == "(":
                stack.pop()  # Remove the matching '('
    while stack:
        c = stack.pop()
        if c in math_operators:  # Only append operators
            if result and result[-1] != " ":  # Ensure space before operator
                result += " "
            result += c
    return result


def next_math_op(stack):
    while len(stack) > 0:
        c = stack.pop()
        if c in math_operators:
            return " " + c
    return ""


print(to_postfix("((51+2)*(8-3))/4"))
print(to_postfix("a + a * b + b"))

5 1 2 +8 3 - *4 /
a a b b + * +


In [13]:
def test_to_postfix():
    """
    Test function for the provided to_postfix function.
    Compares output to expected postfix from a standard algorithm.
    """
    # Define test cases: (Infix Expression, Expected Postfix, Description)
    # Note: Expected Postfix is based on a CORRECT infix-to-postfix algorithm (Shunting-Yard),
    # NOT necessarily what the provided function will output due to its limitations.
    test_cases = [
        ("a+b", "a b +", "Simple addition"),
        ("a*b+c", "a b * c +", "Precedence: multiplication before addition"),
        ("a+b*c", "a b c * +", "Precedence: multiplication before addition"),
        ("(a+b)*c", "a b + c *", "Parentheses overriding precedence"),
        ("a+(b*c)", "a b c * +", "Parentheses confirming precedence"),
        ("a+b/c-d", "a b c / + d -", "Multiple operators, left associativity"),
        ("a/b/c", "a b / c /", "Left associativity of division"),
        ("a-b+c", "a b - c +", "Left associativity of subtraction/addition"),
        ("a^b*c", "a b ^ c *", "Exponentiation precedence"),
        ("a*b^c", "a b c ^ *", "Exponentiation precedence"),
        (
            "a^b^c",
            "a b c ^ ^",
            "Right associativity of exponentiation",
        ),  # This will fail due to lack of associativity logic
        (
            "a^(b^c)",
            "a b c ^ ^",
            "Right associativity with parentheses",
        ),  # This will fail
        (
            "((5+2)*(8-3))/4",
            "5 2 + 8 3 - * 4 /",
            "Complex expression",
        ),  # This will likely fail badly due to char processing/logic
        (
            "51+2",
            "51 2 +",
            "Multi-digit number test",
        ),  # This will fail due to character processing
        (
            "a1+b2",
            "a1 b2 +",
            "Multi-character variable test",
        ),  # This will fail due to character processing
        (
            "10+20*30",
            "10 20 30 * +",
            "Multiple digit numbers and precedence",
        ),  # Will fail multi-digit and precedence
        ("a", "a", "Single operand"),
        ("123", "123", "Single number"),  # Will fail due to character processing
        (" ", "", "Empty/whitespace input"),  # Should ideally return empty string
    ]

    print("--- Testing to_postfix function ---")
    all_passed = True
    for infix_expression, expected_postfix, description in test_cases:
        actual_postfix = to_postfix(infix_expression)

        # Use strip() and potentially split/join for comparison due to potential spacing issues
        # However, showing the raw output is important to highlight spacing problems.
        if actual_postfix.strip() != expected_postfix.strip():
            print(f"FAIL: {description}")
            print(f"  Infix:    '{infix_expression}'")
            print(f"  Expected: '{expected_postfix}'")
            print(f"  Actual:   '{actual_postfix}'")
            all_passed = False
        else:
            print(f"PASS: {description}")
            # Optional: uncomment below to see details of passing tests
            # print(f"  Infix:    '{infix_expression}'")
            # print(f"  Actual:   '{actual_postfix}'")

    if all_passed:
        print("\nAll tests passed!")
    else:
        print(
            "\nSome tests failed. The provided function does not fully implement the standard infix-to-postfix algorithm."
        )
        print("Key issues include:")
        print(
            "- Processing character by character (fails for multi-digit numbers/variables)"
        )
        print("- Ignoring operator precedence")
        print(
            "- Ignoring operator associativity (especially right-associativity for '^')"
        )
        print("- Incorrect handling of parentheses (only pops one operator per ')'")
        print("- Inconsistent spacing in output")
        print(
            "\nConsider implementing the standard Shunting-Yard algorithm for correctness."
        )


# Run the tests
test_to_postfix()

# Run the user's original examples
print("\n--- Running original examples ---")
print(f"to_postfix('((51+2)*(8-3))/4'): '{to_postfix('((51+2)*(8-3))/4')}'")
print(f"to_postfix('a + a * b + b'): '{to_postfix('a + a * b + b')}'")

--- Testing to_postfix function ---
PASS: Simple addition
FAIL: Precedence: multiplication before addition
  Infix:    'a*b+c'
  Expected: 'a b * c +'
  Actual:   'a b c + *'
PASS: Precedence: multiplication before addition
FAIL: Parentheses overriding precedence
  Infix:    '(a+b)*c'
  Expected: 'a b + c *'
  Actual:   'a b +c *'
PASS: Parentheses confirming precedence
FAIL: Multiple operators, left associativity
  Infix:    'a+b/c-d'
  Expected: 'a b c / + d -'
  Actual:   'a b c d - / +'
FAIL: Left associativity of division
  Infix:    'a/b/c'
  Expected: 'a b / c /'
  Actual:   'a b c / /'
FAIL: Left associativity of subtraction/addition
  Infix:    'a-b+c'
  Expected: 'a b - c +'
  Actual:   'a b c + -'
FAIL: Exponentiation precedence
  Infix:    'a^b*c'
  Expected: 'a b ^ c *'
  Actual:   'a b c * ^'
PASS: Exponentiation precedence
PASS: Right associativity of exponentiation
PASS: Right associativity with parentheses
FAIL: Complex expression
  Infix:    '((5+2)*(8-3))/4'
  Expect

## Problem 9.7
Implement a queue using a singly linked list.

In [20]:
class LinkedQueue:
    class _Node:
        __slots__ = "_element", "_next"  # Optimize memory usage

        def __init__(self, element, next):
            self._element = element
            self._next = next

    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def enqueue(self, element):
        newest = self._Node(element, None)
        if self.is_empty:
            self._head = newest
        else:
            self._tail._next = newest
        self._tail = newest
        self._size += 1

    def dequeue(self):
        if self.is_empty:
            raise Exception("Queue is empty")
        answer = self._head._element
        self._head = self._head._next
        self._size -= 1
        if self.is_empty:
            self._tail = None
        return answer

    def first(self):
        if self.is_empty:
            raise Exception("Queue is empty")
        return self._head._element

## Problem 9.8

### R-7.5

Implement a function that counts the number of nodes in a circularly linked list.


In [30]:
class CircularlyLinkedList:
    """Lightweight circularly linked list implementation."""

    class _Node:
        """Lightweight, nonpublic class for storing a node."""

        __slots__ = "_element", "_next"  # Optimize memory usage

        def __init__(self, element, next):
            self._element = element
            self._next = next

    def __init__(self):
        """Create an empty list."""
        self._tail = None  # will be None for empty list
        self._size = 0  # number of elements in the list

    def __len__(self):
        """Return the number of elements in the list."""
        return self._size

    def is_empty(self):
        """Return True if the list is empty."""
        return self._size == 0

    def add_first(self, element):
        """Add an element to the front of the list."""
        newest = self._Node(element, None)  # Node created, next is initially None

        if self.is_empty():
            newest._next = newest  # Link to itself if list becomes size 1
            self._tail = newest  # New node becomes the tail
        else:
            newest._next = self._tail._next  # New node points to current head
            self._tail._next = newest  # Tail points to new node (making it head)

        self._size += 1

    def add_last(self, element):
        """Add an element to the back of the list."""
        newest = self._Node(element, None)  # Node created, next is initially None

        if self.is_empty():
            newest._next = newest  # Link to itself if list becomes size 1
            self._tail = newest  # New node becomes the tail
        else:
            newest._next = self._tail._next  # New node points to current head
            self._tail._next = newest  # Current tail points to new node
            self._tail = newest  # New node becomes the tail

        self._size += 1

    def remove_first(self):
        """Remove and return the first element of the list.

        Raise IndexError if the list is empty.
        """
        if self.is_empty():
            raise IndexError("List is empty")

        head = self._tail._next
        element = head._element

        if self._size == 1:
            self._tail = None  # List becomes empty
        else:
            self._tail._next = head._next  # Bypass the head node

        self._size -= 1
        return element


# SELF IMPLEMENTED FOR TASK
def count_nodes(circ_linked_list):
    if circ_linked_list.is_empty():
        return 0
    current = circ_linked_list._tail._next
    counter = 0
    start_node = current
    while True:
        counter += 1
        current = current._next
        if current == start_node:
            break
    return counter


cll = CircularlyLinkedList()
cll.add_first(10)
print(count_nodes(cll))
cll.add_last(20)
print(count_nodes(cll))
cll.add_last(20)
print(count_nodes(cll))
cll.add_last(20)
cll.add_first(20)
cll.add_last(20)
cll.add_first(20)
print(count_nodes(cll))


1
2
3
7


### R-7.9
Give a fast algorithm for concatenating two doubly linked lists L and M, with header and trailer sentinel nodes, into a single list L′.

In [31]:
def concat_dll(dll1, dll2):
    if dll2.is_empty():
        return dll1
    if not dll1.is_empty():
        dll1._tail._next = dll2._head
        dll2._head._prev = dll1._tail
        dll1._tail = dll2._tail
        dll1._size += dll2._size
        return dll1
    else:
        return dll2